In [52]:
#import statements
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from nltk.stem.snowball import SnowballStemmer
#spark ML imports
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer, CountVectorizer, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import VectorAssembler
%matplotlib inline
spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [7]:
#check to see if thata is available in HDFS
!hdfs dfs -ls /user/akarshsahu/data/Netflix_Data/processed_data_final

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 21 items
-rw-r--r--   3 akarshsahu akarshsahu          0 2020-03-06 00:33 /user/akarshsahu/data/Netflix_Data/processed_data_final/_SUCCESS
-rw-r--r--   3 akarshsahu akarshsahu  958629256 2020-03-06 00:33 /user/akarshsahu/data/Netflix_Data/processed_data_final/part-00000-0163602e-89c0-4771-a723-a9b6ba752b35-c000.csv
-rw-r--r--   3 akarshsahu akarshsahu  919756335 2020-03-06 00:34 /user/akarshsahu/data/Netflix_Data/processed_data_final/part-00001-0163602e-89c0-4771-a723-a9b6ba752b35-c000.csv
-rw-r--r--   3 akarshsahu akarshsahu  814855195 2020-03-06 00:33 /user/akarshsahu/data/Netflix_Data/processed_data_final/part-00002-0163602e-89c0-4771-a723-a9b6ba752b35-c000.csv
-rw-r--r--   3 akarshsahu akarshsahu  879266393 2020-03-06 00:34 /user/akarshsahu/data/Netflix_Data/processed_data_final/part-00003-0163602e-89c0-4771-a723-a9b6ba752b35-c000.csv
-rw-r--r--   3 akarshsahu akarshsahu  9

In [30]:
directory = '/user/akarshsahu/data/Netflix_Data/processed_data_final/'

part_files = 'part-000*-0163602e-89c0-4771-a723-a9b6ba752b35-c000.csv'

final_path = directory + part_files

In [31]:
final_path

'/user/akarshsahu/data/Netflix_Data/processed_data_final/part-000*-0163602e-89c0-4771-a723-a9b6ba752b35-c000.csv'

In [95]:
df = spark.read.csv(final_path)

In [34]:
df.count()

79000155

In [71]:
from pyspark.sql.functions import col, countDistinct
df.agg(countDistinct(col("_c0")).alias("count")).show()

+-----+
|count|
+-----+
| 7602|
+-----+



In [97]:
old_names = df.schema.names
new_names = ['Movie_Id','Cust_Id', 'Rating', 'tconst','Year','Name','primaryTitle','titleType','runtimeMinutes','startYear','isAdult',\
'genres','averageRating','numVotes','Actor_0','Actor_1','Actor_2','Actor_3','Actor_4','Actor_5','Actor_6','Actor_7','Actor_8','Actor_9',\
'Director_0','Director_1','Director_2','Director_3','Director_4','Director_5']

In [98]:
for i in range(len(old_names)):
    print(new_names[i])
    df = df.withColumnRenamed(old_names[i], new_names[i])

Movie_Id
Cust_Id
Rating
tconst
Year
Name
primaryTitle
titleType
runtimeMinutes
startYear
isAdult
genres
averageRating
numVotes
Actor_0
Actor_1
Actor_2
Actor_3
Actor_4
Actor_5
Actor_6
Actor_7
Actor_8
Actor_9
Director_0
Director_1
Director_2
Director_3
Director_4
Director_5


In [99]:
df = df.withColumn('Movie_Id', df['Movie_Id'].cast(IntegerType()))
df = df.withColumn('Cust_Id', df['Cust_Id'].cast(IntegerType()))
df = df.withColumn('Rating', df['Rating'].cast(DoubleType()))
df = df.withColumn('tconst', df['tconst'].cast(StringType()))
df = df.withColumn('Year', df['Year'].cast(IntegerType()))
df = df.withColumn('Name', df['Name'].cast(StringType()))
df = df.withColumn('primaryTitle', df['primaryTitle'].cast(StringType()))
df = df.withColumn('titleType', df['titleType'].cast(StringType()))
df = df.withColumn('runtimeMinutes', df['runtimeMinutes'].cast(DoubleType()))
df = df.withColumn('startYear', df['startYear'].cast(IntegerType()))
df = df.withColumn('isAdult', df['isAdult'].cast(IntegerType()))
df = df.withColumn('genres', df['genres'].cast(StringType()))
df = df.withColumn('averageRating', df['averageRating'].cast(DoubleType()))
df = df.withColumn('numVotes', df['numVotes'].cast(IntegerType()))
df = df.withColumn('Actor_0', df['Actor_0'].cast(StringType()))
df = df.withColumn('Actor_1', df['Actor_1'].cast(StringType()))
df = df.withColumn('Actor_2', df['Actor_2'].cast(StringType()))
df = df.withColumn('Actor_3', df['Actor_3'].cast(StringType()))
df = df.withColumn('Actor_4', df['Actor_4'].cast(StringType()))
df = df.withColumn('Actor_5', df['Actor_5'].cast(StringType()))
df = df.withColumn('Actor_6', df['Actor_6'].cast(StringType()))
df = df.withColumn('Actor_7', df['Actor_7'].cast(StringType()))
df = df.withColumn('Actor_8', df['Actor_8'].cast(StringType()))
df = df.withColumn('Actor_9', df['Actor_9'].cast(StringType()))
df = df.withColumn('Director_0', df['Director_0'].cast(StringType()))
df = df.withColumn('Director_1', df['Director_1'].cast(StringType()))
df = df.withColumn('Director_2', df['Director_2'].cast(StringType()))
df = df.withColumn('Director_3', df['Director_3'].cast(StringType()))
df = df.withColumn('Director_4', df['Director_4'].cast(StringType()))
df = df.withColumn('Director_5', df['Director_5'].cast(StringType()))


In [100]:
df.printSchema()

root
 |-- Movie_Id: integer (nullable = true)
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- tconst: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- runtimeMinutes: double (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)
 |-- Actor_0: string (nullable = true)
 |-- Actor_1: string (nullable = true)
 |-- Actor_2: string (nullable = true)
 |-- Actor_3: string (nullable = true)
 |-- Actor_4: string (nullable = true)
 |-- Actor_5: string (nullable = true)
 |-- Actor_6: string (nullable = true)
 |-- Actor_7: string (nullable = true)
 |-- Actor_8: string (nullable = true)
 |-- Actor_9: string (nullable = true)
 |-- Director_0: string (nullable = tr

In [101]:
df.limit(2).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Actor_6,Actor_7,Actor_8,Actor_9,Director_0,Director_1,Director_2,Director_3,Director_4,Director_5
0,3,1025579,4.0,tt0119448,1997,Character,Character,movie,122.0,1997,...,None,None,None,None,Mike van Diem,None,None,None,None,None
1,3,712664,5.0,tt0119448,1997,Character,Character,movie,122.0,1997,...,None,None,None,None,Mike van Diem,None,None,None,None,None


In [102]:
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

In [112]:
df.write.mode('overwrite').saveAsTable("aghose.processed_dataset")

In [113]:
df1 = hive_context.table("aghose.processed_dataset")

In [114]:
df1.limit(10).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Actor_6,Actor_7,Actor_8,Actor_9,Director_0,Director_1,Director_2,Director_3,Director_4,Director_5
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
2,7706,250166,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
3,7706,328654,3.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
4,7706,2414873,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
5,7706,1239283,2.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
6,7706,1057021,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
7,7706,2633985,3.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
8,7706,2419562,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
9,7706,1047788,2.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
